In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib

import logging
import sys, os

date_strftime_format = '%Y-%m-%d %H:%M:%S'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.join(os.getcwd(), 'credential.json')


from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

logging.basicConfig(stream=sys.stdout, level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt=date_strftime_format)

%load_ext autotime

time: 44.6 μs (started: 2025-08-19 08:40:28 -04:00)


In [2]:
import market_data.ingest.common
import market_data.machine_learning.resample.calc
import market_data.util
import market_data.util.time
from market_data.feature.label import FeatureLabel, FeatureLabelCollection
from market_data.feature.param import SequentialFeatureParam
from market_data.ingest.common import AGGREGATION_MODE, DATASET_MODE, EXPORT_MODE, CacheContext
from market_data.machine_learning.ml_data.cache import calculate_and_cache_ml_data, load_cached_ml_data
from market_data.machine_learning.ml_data.calc import calculate as calculate_ml_data
from market_data.machine_learning.resample.calc import CumSumResampleParams
from market_data.target.param import TargetParams, TargetParamsBatch

cache_context = market_data.ingest.common.CacheContext(
    DATASET_MODE.OKX, EXPORT_MODE.BY_MINUTE, AGGREGATION_MODE.TAKE_LATEST)


time: 325 ms (started: 2025-08-19 08:40:28 -04:00)


In [8]:
time_range = market_data.util.time.TimeRange(
    date_str_from='2025-06-30', date_str_to='2025-07-01',
    )

target_params_batch = TargetParamsBatch(
        target_params_list=[TargetParams(forward_period=int(period), tp_value=float(tp), sl_value=float(tp)) 
            for period in [5, 10, 30]
            for tp in [0.015, 0.03, 0.05]]
        )


ml_data = calculate_ml_data(
    CacheContext(market_data.ingest.common.DATASET_MODE.OKX, market_data.ingest.common.EXPORT_MODE.BY_MINUTE, market_data.ingest.common.AGGREGATION_MODE.TAKE_LATEST),
    time_range=time_range,
    feature_collection = FeatureLabelCollection(),
    target_params_batch=target_params_batch,
    resample_params=CumSumResampleParams(price_col = 'close', threshold = 0.1),
)

2025-08-19 23:25:12 - INFO - Loading regular feature_resampled data for 0 features
2025-08-19 23:25:12 - INFO - Successfully prepared regular ML data with 38 rows and 59 columns
time: 542 ms (started: 2025-08-19 23:25:12 -04:00)


In [9]:
ml_data.shape

(38, 59)

time: 1.67 ms (started: 2025-08-19 23:25:12 -04:00)
